In [1]:
#from exploringShipLogbooks.classification import LogbookClassifier

import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

#import rpy2
#from rpy2.robjects import pandas2ri
#pandas2ri.activate()

from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
#from exploringShipLogbooks.basic_utils import MultiColumnLabelEncoder
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import clean_data

from exploringShipLogbooks.config import *


# Load and clean data
### Load CLIWOC ship logs

In [ ]:
# extract data from zip file
cliwoc_data = extract_logbook_data('CLIWOC15.csv')


### Find slave training data in CLIWOC data set

In [ ]:
# extract logs that mention slaves
mentions_slaves = wc.count_key_words(cliwoc_data, text_columns, slave_words)
slave_index = mentions_slaves[(mentions_slaves['ContainsKeyWord'] != 0)].index

# cleanup
del mentions_slaves

### Clean CLIWOC data

In [ ]:
# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

# clean data (make all same case)
cliwoc_data = clean_data(cliwoc_data)

### Load Slave Voyages data

In [ ]:
#filename = './exploringShipLogbooks/data/tastdb-exp-2010.sav'
#slave_voyage_logs = rpy2.robjects.r('foreign::read.spss("%s", to.data.frame=TRUE)' % filename)



### Clean Slave voyages data

In [ ]:
#slave_voyage_desired_cols = ['portdep', 'portret', 'shipname', 'rig', 'national', 'yeardep']
#slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

#slave_voyage_logs.columns = ['ShipName', 'Nationality', 'ShipType', 'VoyageFrom', 'VoyageTo', 'Year']

#slave_voyage_logs = clean_data(slave_voyage_logs)

# Join data sets

In [ ]:
all_data = cliwoc_data # .append(slave_voyage_logs, ignore_index = True)

del cliwoc_data

# Encode data
- Must encode data before separating, otherwise values that do not occur in a subset will be encoded differently

In [ ]:
all_data = encode_data_df(all_data)

# Extract training data, and create list of classes

In [ ]:
# split log data into data that mentions slaves and data that does not
# ** will add new data set to this once I can use R!
slave_logs = all_data.loc[slave_index.values]

# ****arbitrarily using first 1000 logs as "no slave" training data.
# need to figure out what to put here.
no_slave_index = range(1000)
no_slave_logs = all_data.loc[no_slave_index]

# remaining data is unclassified 
unclassified_logs = all_data.drop(slave_index)
unclassified_logs = all_data.drop(no_slave_index)

# clean-up
del cliwoc_data

In [ ]:
# create list of classes for training data (0 is for non-slave, 1 is for slave)
classes = [0] * len(cliwoc_no_slave_logs)
classes.append([1] * len(cliwoc_slave_logs))

# joint training data
training_data = cliwoc_no_slave_logs.append(cliwoc_slave_logs, ignore_index=True)



# Fit training data to classifier

In [ ]:

classifier = MultinomialNB()
classifier.fit(original_data, original_classes)

MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
prediction = classifier.predict(original_data)

print('Predictions for original data',
      '(should be same as original classes): \n', prediction)

new_data = np.random.randint(5, size = (1,100))
prediction = classifier.predict(new_data)

print('\n Predictions for new data (random)'
      ', should change for different runs: \n', prediction)